## RNN
이번에는 Day4에서 했던 MNIST데이터셋을 **신경망**으로 학습시켰던 것을 이번에는 **RNN**을 이용해 학습시켜봅시다.   
앞서 설명했듯이 RNN의 구조상 학습데이터를 단계별로 구분하여 입력해야 합니다. 즉 입력 데이를 단계별로 쪼개어 주어야 한다는 의미이므로 MNIST의 데이터 역시 쪼개어 줘야 합니다. 우리는 MNIST데이터가 28x28이니 28개로 쪼개 28x1을 28개로 만들어 총 28단계를 거쳐 넣어줄겁니다. 

In [9]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)

learning_rate = 0.001
total_epoch = 30
batch_size = 128

n_input = 28
n_step =28
n_hidden = 128
n_class = 10

X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


필요한 것들을 정의 해줍시다. 기존의 신경망(Day4 MNIST.ipynb)과 달리 RNN에서는 X에 **n_step**이라는 차원하나를 추가해 줍니다.  

 여기서 RNN은 순서가 있는 데이터를 다루므로 한번에 입력 받을 개수와 총 몇 단계로 이뤄진 데이터를 받을지 결정해 줘야 합니다.   
따라서 가로픽셀수 28x1를 **n_input**으로 세로픽셀수(28단계)를 **n_step**으로 설정합니다.   

출력값(**n_class**)은 MNIST의 분류인 0~9까지의 10개의 숫자를 원-핫 인코딩으로 표현하도록 합니다.

In [3]:
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


다음으로 n_hidden(128)개의 출력값을 갖는 RNN 셀을 생성합니다.   


In [4]:
outputs, states = tf.nn.dynamic_rnn(cell,X, dtype = tf.float32)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


**dynamic_rnn**함수를 이용해 RNN신경망을 완성합니다.   
추가로 RNN 신경망에서 나오는 출력값은 각 단계가 포함된 [batch_size, n_step, n_hidden] 형태 입니다.

In [0]:
outputs = tf.transpose(outputs,[1,0,2])  #[batch_size, n_step, n_hidden] -> [n_step, batch_size, n_hidden]
outputs = outputs[-1] #[n_step, batch_size, n_hidden] -> [batch_size, n_hidden]

RNN에서 나온 출력값을 가지고 최종 출력값을 만들어 봅시다.   
tf.nn.softmax함수를 쓰기위해선 최종 결과값이 실측값 Y와 동일한 형태인 [batch_size,n_class]여야 합니다. 따라서 위 함수들을 이용해 outputs를 동일한 형태로 만들어 줍니다.

In [0]:
model = tf.matmul(outputs, W)+b

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

지금까지 만든 모델과 실측값을 비교하여 손실값을 구하고, 신경망을 최적화 하는 함수를 사용하여 신경망 구성을 마무리 합니다.   

이제 앞서 구성한 신경망을 학습시키고 결과를 확인해 봅시다.

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/ batch_size)

for epoch in range(total_epoch):
  total_cost = 0

  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

    _, cost_val = sess.run([optimizer,cost], feed_dict = {X: batch_xs, Y: batch_ys}) ##파이썬 인터프리터에선 마지막으로 실행된 결과값(batch_xs)이 _라는 변수에 저장된다.
    total_cost += cost_val

  print('Epoch:', '%04d'%(epoch+1), 'Avg. cost =', '{:.3f}'.format(total_cost/ total_batch))

print('최적화 완료')

is_correct = tf.equal(tf.argmax(model,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict = {X: test_xs, Y: test_ys}))

Epoch: 0001 Avg. cost = 0.480
Epoch: 0002 Avg. cost = 0.210
Epoch: 0003 Avg. cost = 0.170
Epoch: 0004 Avg. cost = 0.148
Epoch: 0005 Avg. cost = 0.133
Epoch: 0006 Avg. cost = 0.125
Epoch: 0007 Avg. cost = 0.123
Epoch: 0008 Avg. cost = 0.109
Epoch: 0009 Avg. cost = 0.102
Epoch: 0010 Avg. cost = 0.102
Epoch: 0011 Avg. cost = 0.102
Epoch: 0012 Avg. cost = 0.094
Epoch: 0013 Avg. cost = 0.089
Epoch: 0014 Avg. cost = 0.086
Epoch: 0015 Avg. cost = 0.079
Epoch: 0016 Avg. cost = 0.087
Epoch: 0017 Avg. cost = 0.083
Epoch: 0018 Avg. cost = 0.072
Epoch: 0019 Avg. cost = 0.078
Epoch: 0020 Avg. cost = 0.079
Epoch: 0021 Avg. cost = 0.070
Epoch: 0022 Avg. cost = 0.071
Epoch: 0023 Avg. cost = 0.069
Epoch: 0024 Avg. cost = 0.070
Epoch: 0025 Avg. cost = 0.071
Epoch: 0026 Avg. cost = 0.066
Epoch: 0027 Avg. cost = 0.076
Epoch: 0028 Avg. cost = 0.072
Epoch: 0029 Avg. cost = 0.063
Epoch: 0030 Avg. cost = 0.060
최적화 완료
정확도: 0.9696
